# Data Dictionary

|Feature|Type|Description|
|---|---|---|
|Product_Code|*string*|The product name encoded|
|Warehouse|*string*|Warehouse ID|
|Product_Category|*string*|Product category of each product|
|Date|*string*|The date customer needs the product|
|Order_Demand|*string*|Order quantity|

In [147]:
# Import libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy.random import seed
from numpy import array
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow_addons.metrics import RSquare
from tensorflow.keras.callbacks import EarlyStopping

In [148]:
# Read csv file and display shape.
df = pd.read_csv('../Data_Historical/df2.csv').iloc[:,1:]

# Get list of product codes.
products = df['Product_Code'].unique()
df.shape

(924272, 5)

In [149]:
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0965,Whse_A,Category_006,2011-01-08,2
1,Product_1724,Whse_A,Category_003,2011-05-31,108
2,Product_1521,Whse_S,Category_019,2011-06-24,85000
3,Product_1521,Whse_S,Category_019,2011-06-24,7000
4,Product_1507,Whse_C,Category_019,2011-09-02,1250


In [150]:
# Function to clean and resample order demand.
def get_product(data,product_code):
    df_init = data[data['Product_Code']==product_code]
    df_init['Date'] = pd.to_datetime(df_init['Date'])
    df_init = df_init.set_index('Date')
    df_out = pd.DataFrame(df_init['Order_Demand'].resample('W').sum())
    df_out.rename(columns={'Order_Demand':'demand'+'_'+product_code[-4:]}, inplace=True)
    return df_out

# split a univariate sequence
def split_sequence(sequence, n_steps):
 X, y = list(), list()
 
 for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
        break
    
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
 return array(X), array(y)

# Function to apply standard scaling to input.
def scale(X_train): #, X_test
 # fit scaler
 scaler = StandardScaler()
 # Fit & transform train
 X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
 # Transform test
 #X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
 return scaler, X_train #, X_test

# Function to reshape and scale the previous week's sales.
def reshape_scale(y0):
    # Reshape to (1,1,1)
    wk0 = y0[-1:].reshape((1,1,n_features))
    # Transform to standard scaling
    wk0 = scaler.transform(wk0.reshape(-1, wk0.shape[-1])).reshape(wk0.shape)
    return wk0

# LSTM Model.
def fit_lstm (X, y):
    model = Sequential()
    model.add(LSTM(64, activation='relu', return_sequences=True,  input_shape=(n_steps, n_features))) 
    model.add(LSTM(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=.001), loss='mse')
    lstm = model.fit(X, y, epochs=800, verbose=0, batch_size=32)
    return model, lstm

In [151]:
# Function to run the model and predictions
def run(df, product):
    n_steps = 1
    n_features = 1
    preds = list()   
    # Create dataframe of Product
    prod = get_product(df, product)
    # define input sequence
    raw_seq = prod.iloc[:,0]
    # split into samples
    X, y = split_sequence(raw_seq, n_steps)
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    # Standard scaling X
    scaler, X_scaled = scale(X)
    model, lstm = fit_lstm(X_scaled, y)
    wk0 = reshape_scale(y[-1:])
    # Predict Week 1
    predwk1 = model.predict(wk0)
    preds.append(predwk1)
    # Predict Week 2
    predwk2 = model.predict(reshape_scale(predwk1))
    preds.append(predwk2)
    # Predict Week 3
    predwk3 = model.predict(reshape_scale(predwk2))
    preds.append(predwk3)
    # Predict Week 4
    predwk4 = model.predict(reshape_scale(predwk3))
    preds.append(predwk4)
    # Reshape predictions to 2 D
    reshaped = np.reshape(preds, (4,))
    return reshaped



# define input sequence
raw_seq = prod_1359['demand_1359']

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], n_features))

# Standard scaling X
scaler, X_scaled = scale(X)

# Final Model - LSTM

model, lstm = fit_lstm(X_scaled, y)

wk0 = reshape_scale(y[-1:])
predwk1 = model.predict(wk0)
predwk2 = model.predict(reshape_scale(predwk1))
predwk3 = model.predict(reshape_scale(predwk2))
predwk4 = model.predict(reshape_scale(predwk3))

prod_list = list()
preds = list()
for product in products:
    # Create dataframe of Product
    prod = get_product(df, product)
    # define input sequence
    raw_seq = prod.iloc[:,0]
    # split into samples
    X, y = split_sequence(raw_seq, n_steps)
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    # Standard scaling X
    scaler, X_scaled = scale(X)
    model, lstm = fit_lstm(X_scaled, y)
    wk0 = reshape_scale(y[-1:])
    predwk1 = model.predict(wk0)
    prod_list.append(product)
    preds.append(predwk1)
    predwk2 = model.predict(reshape_scale(predwk1))
    prod_list.append(product)
    preds.append(predwk2)
    predwk3 = model.predict(reshape_scale(predwk2))
    prod_list.append(product)
    preds.append(predwk3)
    predwk4 = model.predict(reshape_scale(predwk3))
    prod_list.append(product)
    preds.append(predwk4)
    print (product)

In [152]:
dfs = pd.DataFrame()
for product in products[0:1080]:
    predictions = run(df, product)
    df_prod = pd.DataFrame({product:predictions})
    dfs = pd.concat([dfs, df_prod],axis=1)
dfs.astype(int)
    

C:\Users\Zeonice\AppData\Local\Temp\ipykernel_16572\1616143787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['Date'] = pd.to_datetime(df_init['Date'])


1/1 [==============================] - 0s 29ms/step


C:\Users\Zeonice\AppData\Local\Temp\ipykernel_16572\1616143787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['Date'] = pd.to_datetime(df_init['Date'])


1/1 [==============================] - 0s 27ms/step


C:\Users\Zeonice\AppData\Local\Temp\ipykernel_16572\1616143787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['Date'] = pd.to_datetime(df_init['Date'])


...

1/1 [==============================] - 0s 29ms/step


C:\Users\Zeonice\AppData\Local\Temp\ipykernel_16572\1616143787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['Date'] = pd.to_datetime(df_init['Date'])


1/1 [==============================] - 0s 31ms/step


C:\Users\Zeonice\AppData\Local\Temp\ipykernel_16572\1616143787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['Date'] = pd.to_datetime(df_init['Date'])


KeyboardInterrupt: 